# template-project demo

The purpose of this notebook is to demonstrate the functionality of `template-project`.

The demo is organised to show

- Step 1: Explanation

- Step 2: Explanation

Note that when you submit a pull request, you should `clear all outputs` from your python notebook for a cleaner merge.


In [ ]:
from pathlib import Path

from template_project import plotters, readers, tools, utilities, writers


In [ ]:
ds = readers.load_sample_dataset()

In [ ]:
plotters.plot_monthly_transport(ds)

In [ ]:
path = Path.cwd().parent / "data" / "temp.nc"
writers.save_dataset(ds, output_file=path, prompt_user=True)